In [1]:
# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import json
import pandas as pd
import os
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
# from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [2]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"
model_dir = 'model_esxdeploy_classifier_target_distilbert_1'
path_data = 'datasource/esxdeploy_0512_selected_validate_nodup_label_merged_predicted.json'
max_length = 512

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
def load_content(path_data):
    with open(path_data, 'r') as file:
        dict_filename_content = json.load(file)
    # end
    
    return dict_filename_content
# edn

In [5]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if os.path.exists(model_dir) and len(os.listdir(model_dir)) > 0:
    print('load model from local')
    model_info = model_dir
    with open(os.path.join(model_dir, 'labels.json'), 'r') as file:
        target_names = json.load(file)
    # end
else:
    print('load model from official')
    model_info = model_name
# end

model = DistilBertForSequenceClassification.from_pretrained(model_info, num_labels=len(target_names))
model = model.cuda()

load model from local


In [6]:
contents = load_content(path_data)
list_conf_output = []
list_label_output = []
list_doc_output = []utput for label_output, label_origin in zip(list_label_output, list_label_origin) if label_output == label_origin]) / len(li
list_label_origin = []

for content in contents:

    list_log = content['logs']
    ancher = content['ancher']
    
    list_conf = content['logs_conf_raw']
    list_label = content['logs_label']
    
    ids_relevant = [id for id, label in enumerate(list_label) if label == 'relevant']
    ids_relevant_2 = [id_relevant for id_relevant in ids_relevant if list_conf[id_relevant][1] > 0.8]
    
    logs_relevant = [list_log[id_relevant] for id_relevant in ids_relevant_2]
    logs_relevant_cut = logs_relevant[-9:]
    document = ' '.join(logs_relevant_cut + [ancher])
    
    input_tokenized = tokenizer.encode_plus(document, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to('cuda')
    with torch.no_grad():
        out = model(**input_tokenized, output_hidden_states=True, output_attentions=True)
    # end

    probas_evaluate = torch.nn.functional.softmax(out.logits, dim=-1)
    answer_evaluate = int(probas_evaluate.argmax().cpu())

    label_evaluate = target_names[answer_evaluate]

    list_doc_output.append(document)
    list_conf_output.append(probas_evaluate.cpu().numpy().tolist()[0][answer_evaluate])
    list_label_output.append(label_evaluate)
    list_label_origin.append(content['target'])
# end

In [7]:
contents = load_content(path_data)
list_label_origin = [content['target'] for content in contents]

In [8]:
len(list_label_origin)

267

In [10]:
len([label_output for label_output, label_origin in zip(list_label_output, list_label_origin) if label_output == label_origin]) / len(list_label_origin)

0.6404494382022472